# Association Bounding Boxes with the Hungarian Algorithm

The general idea is to associate bounding boxes between successive frames in a video. This will provide the ability to not just detect, but also track objects. The `linear_sum_assignment` function in the `scipy.optimize` module implements the actual Hungarian Algorithm.

In [1]:
import random
from scipy.optimize import linear_sum_assignment
import numpy as np

# Detections at timestep N - 1.
A = [100, 120, 130, 330]
B = [300, 350, 400, 400]
C = [577, 138, 709, 244]

# Detections at timestep N
D = [50, 400, 100, 550]  # Should match no frame.
E = [99, 120, 132, 333]  # Should match frame A.
F = [302, 352, 406, 400] # Should match frame B.

old = [A,B,C]
new = [D,E,F]
print(old)
print(new)

[[100, 120, 130, 330], [300, 350, 400, 400], [577, 138, 709, 244]]
[[50, 400, 100, 550], [99, 120, 132, 333], [302, 352, 406, 400]]


## Using the IOU Metric

Compute the intersection-over-union (IOU) metric for each pair of bounding boxes, then use these values at input to the Hungarian Algorithm matrix.

1. Create a matrix and store the IOU for all boxes.
2. Apply the Hungarian Algorithm.
3. Identify false positives and false negatives.

In [2]:
def compute_iou(box_1, box_2):
    
    """
    Computes the IOU metric for a pair of bounding boxes.
    """
    
    xA = max(box_1[0], box_2[0])
    yA = max(box_1[1], box_2[1])
    xB = min(box_1[2], box_2[2])
    yB = min(box_1[3], box_2[3])
    inter_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    
    # Calculate Union(A,B) = A + B - Inter(A,B)
    box1_area = (box_1[2] - box_1[0] + 1) * (box_1[3] - box_1[1] + 1)
    box2_area = (box_2[2] - box_2[0] + 1) * (box_2[3] - box_2[1] + 1)
    union_area = (box1_area + box2_area) - inter_area

    # Compute and return the IOU metric.
    return inter_area / float(union_area)

In [3]:
# Compute the IOU matrix for the old boxes and new boxes.

iou_matrix = np.zeros((len(old), len(new)), dtype=np.float32)

for i, old_box in enumerate(old):

    for j, new_box in enumerate(new):

        iou_matrix[i][j] = compute_iou(old_box, new_box)

print(iou_matrix)

[[0.         0.89898294 0.        ]
 [0.         0.         0.8909091 ]
 [0.         0.         0.        ]]


In [4]:
# Go through the IOU matrix and replace positive values with 1.0, always
# take the maximum value (if there are two positive values).

for idx, iou in enumerate(iou_matrix):

    iou_matrix[idx] = [1 if (x == max(iou) and max(iou) > 0) else 0 for x in iou]

print("Match Matrix")
print(iou_matrix)

Match Matrix
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]]


In [5]:
# Invoke the Linear Assignment Method (Hungarian Algorithm)

hungarian_row, hungarian_col = linear_sum_assignment(-iou_matrix)

print("Hungarian Matrix")
print(hungarian_row)
print(hungarian_col)

Hungarian Matrix
[0 1 2]
[1 2 0]


In [6]:
# Create lists for matches, unmatched detections, and unmatched trackings.

matches, unmatched_trackers, unmatched_detections = [], [], []

In [7]:
# Reshape the hungarian matrix to make it easier to use.

hungarian = np.array(list(zip(hungarian_row, hungarian_col)))

print(hungarian)

[[0 1]
 [1 2]
 [2 0]]


In [8]:
for h in hungarian:
    
    if(iou_matrix[h[0], h[1]] < 0.3):
        
        unmatched_trackers.append(old[h[0]])
        unmatched_detections.append(new[h[1]])
        
    else:
        
        matches.append(h.reshape(1, 2))
    
if(len(matches) == 0):

    matches = np.empty((0, 2), dtype=int)
    
else:
    
    matches = np.concatenate(matches, axis=0)

print("Matches")
print(matches)
print("Unmatched Detections")
print(unmatched_detections)
print("Unmatched Trackers")
print(unmatched_trackers)

Matches
[[0 1]
 [1 2]]
Unmatched Detections
[[50, 400, 100, 550]]
Unmatched Trackers
[[577, 138, 709, 244]]


In [9]:
for t, trk in enumerate(old):
    
    if(t not in hungarian[:,0]):

        unmatched_trackers.append(t)

for d, det in enumerate(new):

    if(d not in hungarian[:,1]):

        unmatched_detections.append(d)

In [10]:
# Now display the matched bounding boxes.

display_match = []

for matching in matches:

    display_match.append((new[matching[1]], old[matching[0]]))

print("Matched Detections")
print(display_match)
print("Unmatched Detections")
print(np.array(unmatched_detections))
print("Unmatched Trackers")
print(np.array(unmatched_trackers))       

Matched Detections
[([99, 120, 132, 333], [100, 120, 130, 330]), ([302, 352, 406, 400], [300, 350, 400, 400])]
Unmatched Detections
[[ 50 400 100 550]]
Unmatched Trackers
[[577 138 709 244]]
